# 🧠 Mental Wellness Chatbot

## ✅ Required Libraries:

In [9]:
pip install gTTS pygame SpeechRecognition scikit-learn

## ✅ Complete Code with GUI

In [12]:
import json
import random
import time
import threading
import tkinter as tk
from tkinter import scrolledtext

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder

import speech_recognition as sr
from gtts import gTTS
import pygame

# Load dataset
with open("dataset.json", "r") as file:
    data = json.load(file)

# Train model
X, y = [], []
for intent, intent_data in data["intents"].items():
    for example in intent_data["examples"]:
        X.append(example.lower())
        y.append(intent)

le = LabelEncoder()
y_encoded = le.fit_transform(y)
model = make_pipeline(TfidfVectorizer(), LogisticRegression(max_iter=1000))
model.fit(X, y_encoded)

# Functions
def predict_intent(user_input):
    user_input = user_input.lower()
    pred = model.predict([user_input])[0]
    intent = le.inverse_transform([pred])[0]
    return intent

def get_response(intent):
    return data["intents"][intent]["response"]

def speak(text):
    print("Bot:", text)
    tts = gTTS(text)
    tts.save("response.mp3")
    pygame.mixer.init()
    pygame.mixer.music.load("response.mp3")
    pygame.mixer.music.play()
    while pygame.mixer.music.get_busy():
        time.sleep(0.5)
    pygame.mixer.music.unload()

def get_voice_input():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        log_text.insert(tk.END, "🟢 Listening...\n")
        log_text.see(tk.END)
        speak("I'm listening.")
        audio = recognizer.listen(source)

    try:
        user_input = recognizer.recognize_google(audio)
        return user_input
    except sr.UnknownValueError:
        speak("Sorry, I couldn't understand.")
        return ""
    except sr.RequestError:
        speak("Sorry, the speech service is unavailable.")
        return ""

# GUI Function
def start_voice_chat():
    def run_chat():
        user_input = get_voice_input()
        if user_input:
            log_text.insert(tk.END, "You: " + user_input + "\n")
            if any(word in user_input.lower() for word in ["exit", "bye", "quit", "stop"]):
                farewell = get_response("goodbye")
                log_text.insert(tk.END, "Bot: " + farewell + "\n")
                speak(farewell)
                return
            intent = predict_intent(user_input)
            response = get_response(intent)
            log_text.insert(tk.END, "Bot: " + response + "\n")
            speak(response)
            log_text.see(tk.END)
        else:
            log_text.insert(tk.END, "❌ Could not understand. Try again.\n")
            log_text.see(tk.END)

    # Run in thread to avoid freezing GUI
    threading.Thread(target=run_chat).start()

# GUI Setup
window = tk.Tk()
window.title("🧘 Mental Wellness Chatbot")
window.geometry("500x500")
window.config(bg="#f0f8ff")

title_label = tk.Label(window, text="🧠 Mental Wellness Chatbot", font=("Arial", 18, "bold"), bg="#f0f8ff")
title_label.pack(pady=10)

log_text = scrolledtext.ScrolledText(window, wrap=tk.WORD, width=60, height=20, font=("Consolas", 10))
log_text.pack(padx=10, pady=10)

start_btn = tk.Button(window, text="🎙 Speak to Bot", command=start_voice_chat, font=("Arial", 14), bg="#add8e6", fg="black")
start_btn.pack(pady=10)

window.mainloop()

Bot: I'm listening.
Bot: Hello! I'm here to support your mental well-being. How are you feeling today?
Bot: I'm listening.
Bot: I'm really sorry you're feeling this way. You're not alone, and I'm here to listen. Would you like to talk more about it?
Bot: I'm listening.
Bot: Try to take a few deep breaths with me. Inhale... Exhale... You can also try grounding exercises like naming 5 things you see. Want more tips?
Bot: I'm listening.
Bot: I'm always here to listen. You can share anything with me, and Iâ€™ll respond with care. Whatâ€™s on your mind?
